# Helper Function
> Helper functions

In [1]:
# default_exp helper

## Color loggers

In [1]:
# export 

from types import MethodType

class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

class tint(object):
    def __init__(self, cname):
        self.start = getattr(color,cname.upper())
    
    def __call__(self, txt):
        return f"{self.start}{txt}{color.END}"
    
    def __add__(self,txt):
        return self.__call__(txt)
    
    def __or__(self,txt):
        print(self.__call__(txt))
        return self

for c in ["purple","green","red","blue","yellow","bold","underline","cyan"]:
    setattr(color, c, tint(c))

In [2]:
color.purple|"msg1"|"0000123"

msg1
0000123


In [3]:
print(color.blue("123"))

123


## File, log, data, directory handler

In [35]:
# export
import os
from pathlib import Path
import json
import pandas as pd
from datetime import datetime
import torch
import numpy as np
from torch.utils.tensorboard import SummaryWriter

class tracker(object):
    def __init__(self, libname, fname):
        self.libname = libname
        self.fname = fname
        self.home = Path(os.environ['HOME'])
        self.dir = self.home/f".{libname}"
        self.dir.mkdir(exist_ok = True) 
        self.data = self.dir/"data"
        self.data.mkdir(exist_ok = True)
        self.log = self.dir/"log"
        
        self.log.mkdir(exist_ok = True) 
        self.log_path = self.log/self.fname
        self.log_path.mkdir(exist_ok=True)
        
    def __repr__(self):
        return f"<{self.libname}:{self.fname}>"
        
    def mkdir(self, path):
        Path(path).mkdir(exist_ok=True)
        
    def __setitem__(self, fname,dict_):
        with open(self.data/f"{fname}.json","w") as f: f.write(json.dumps(dict_, indent = 2))

    
    def __getitem__(self,fname):
        try:
            return json.loads(open(self.data/f"{fname}.json","r").read())
        except:
            return None
        
    def logging(self,line):
        with open(self.log_file,"a") as f :f.write(line+"\n")
        return self.log_file
        
    def __call__(self,dict_):
        """
        add a dictionary to log
        """
        self.logging(json.dumps(dict_))
        return self
        
    def lines(self):
        return list(json.loads(i) for i in open(self.log_file).read().split("\n")[:-1])
    
    @property
    def ts(self):
        return datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
class emberTracker(tracker):
    def __init__(self, fname):
        super().__init__("torchember",fname)
        self.latest = self.log/f"{fname}_latest"
        self.latest_lines = ""
        self.writer = SummaryWriter(str(self.log/"runs"),max_queue = 1)
        self.marked = {}
        self.mark(init="00")
        
    def mark(self,**kwargs):
        self.marked.update(kwargs)
        file_name = "_".join(f"{k}-{v}" for k,v in self.marked.items())
        self.log_file = self.log_path/f"{file_name}.log"
        self.writer = SummaryWriter(str(self.log/"runs"/file_name),max_queue = 1)
        
    @property
    def log_files(self):
        return os.listdir(self.log_path)
    
    def tb_scalar(self,dict_):
        module = dict_["module"]
        ttype = dict_["ttype"]
        tname = dict_["tname"]
        for col in ["module","ttype","tname"]:
            del dict_[col]
        for k in list(dict_):
            if type(dict_[k]) not in [float,np.float]:
                del dict_[k]
        tag = f"{module}_{ttype}_{tname}"
        self.writer.add_scalars(tag, dict_)
        self.writer.flush()
        
    def __call__(self,dict_):
        """
        add a dictionary to log
        """
#         self.logging(json.dumps(dict_))
        self.tb_scalar(dict_)
        return self
        
    def logging(self,line):
        with open(self.log_file,"a") as f : f.write(","+line)
        self.latest_lines+=(line+"\n")
        return self.log_file
    
    def refresh(self):
        """
        lastest always contain the record of the latest batch
        """
        with open(self.latest,"w") as f :  f.write(self.latest_lines)
        self.latest_lines = ""
        return self.latest
    
    def latest_line_list(self):
        return list(json.loads(i) for i in open(self.latest).read().split("\n")[:-1])
    
    @property
    def latest_df(self):
        return pd.DataFrame(self.latest_line_list())

### Test on the data tracker

In [36]:
etrack = emberTracker("testmodel")

In [37]:
etrack.mark(phase="train")

In [39]:
etrack.writer.log_dir

'/Users/salvor/.torchember/log/runs/init-00_phase-train'

This is how we record the data within other codes

In [25]:
for i in range(10):
    etrack({"col1":i,"module":"Some Model","ttype":"linear","tname":"grad_0","col2":i*2,"col3":"hahahha"})

Preview log file

In [26]:
!cat {etrack.log_path}

cat: /Users/salvor/.torchember/log/testmodel: Is a directory


Read log as dataframe

In [27]:
!rm {etrack.log_path}

rm: /Users/salvor/.torchember/log/testmodel: is a directory


### Save/ Get JSON data

Save dictionary to json data

In [28]:
etrack["about_this_model"]= {
    "property1":1,
    "property2":False,
    "property3":
        {"size":100,
         "speed":200}
    }

Read this meta data from json file

In [29]:
etrack["about_this_model"]["property2"]

False